In [ ]:
from github import Github
from github import Auth
from dotenv import dotenv_values
import pandas as pd
import sys
import subprocess
sys.path.append('../../')
from utils.constants import DATA_DOWNLOAD_DATE
from utils.dataloader import get_repos

config = dotenv_values("../../../.env")
auth = Auth.Token(config['GITHUB_TOKEN'])
gh = Github(auth=auth)

repositories = get_repos()

In [ ]:
for index, row in repositories.iterrows():
    if row['host'] == 'Github':
        repo = gh.get_repo(row['repo'])
        # get the commit of the last commit before the data download date, download the repo at that commit
        commits = repo.get_commits(until=DATA_DOWNLOAD_DATE)
        # get the last commit before the data download date
        if commits.totalCount > 0:
            commit = commits[0]
            print(f"Downloading {row['repo']} at commit {commit.sha}...")
            # clone the repository at that commit
            subprocess.run(['git', 'clone', repo.clone_url, f"../../../data/github/{row['repo'].replace('/', '_')}"])
            # checkout the commit
            subprocess.run(['git', 'checkout', commit.sha], cwd=f"../../../data/github/{row['repo'].replace('/', '_')}")

In [ ]:
# loc count
for index, row in repositories.iterrows():
    if row['host'] == 'Github':
        repo_path = f"../../../data/github/{row['repo'].replace('/', '_')}"
        subprocess.run(['cloc', '.', '--csv', '--out', f'../../../results/csv/general/loc/{row["repo"].replace("/", "_")}.csv'], cwd=repo_path)